# ***6.Visualz-folium-sairauspaaryhmat-2003-2023***
merge/merge1&_avoin-data_suomen_työkyvyttömyyseläkkeensaajat_2003-2023_merge.csv

In [101]:
import folium
import branca.colormap as cm
import pandas as pd

In [102]:
df = pd.read_csv("merge/merge1&_avoin-data_suomen_työkyvyttömyyseläkkeensaajat_2003-2023_merge.csv")
df

,Maakunta,kunta_nro,kunta_nimi,latitude,longitude,sairauspaaryhma,ikaryhma,sukupuoli,saaja_lkm,kuukausi_nro,vuosi
0,Pohjois-Savo,915,Varkaus,62.313951,27.888587,I00-I99 Verenkiertoelinten sairaudet,55-59,Nainen,13,12,2003
1,Pohjois-Pohjanmaa,625,Pyhäjoki,64.465164,24.259197,M00-M99 Tuki- ja liikuntael. sekä sidekudoksen...,50-54,Mies,5,12,2003
2,Pohjois-Pohjanmaa,625,Pyhäjoki,64.465164,24.259197,F00-F99 Mielenterveyden ja käyttäytymisen häiriöt,50-54,Mies,7,12,2003
3,Pohjois-Pohjanmaa,625,Pyhäjoki,64.465164,24.259197,Yhteensä,50-54,Mies,26,12,2003
4,Pohjois-Pohjanmaa,625,Pyhäjoki,64.465164,24.259197,M00-M99 Tuki- ja liikuntael. sekä sidekudoksen...,45-49,Nainen,5,12,2003
...,...,...,...,...,...,...,...,...,...,...,...
748695,Keski-Suomi,892,Uurainen,62.500806,25.438519,F00-F99 Mielenterveyden ja käyttäytymisen häiriöt,60-64,Mies,6,12,2023
748696,Keski-Suomi,892,Uurainen,62.500806,25.438519,Yhteensä,60-64,Mies,26,12,2023
748697,Keski-Suomi,892,Uurainen,62.500806,25.438519,M00-M99 Tuki- ja liikuntael. sekä sidekudoksen...,60-64,Mies,11,12,2023
748698,Keski-Suomi,892,Uurainen,62.500806,25.438519,F00-F99 Mielenterveyden ja käyttäytymisen häiriöt,Tieto puuttuu,Nainen,10,12,2023


In [103]:
df = df[(df['sairauspaaryhma'] != 'Yhteensä')]

In [104]:
all_locations = df.groupby('kunta_nimi').agg(
    latitude=('latitude', 'first'),
    longitude=('longitude', 'first'),
    Maakunta=('Maakunta', 'first'),
    saaja_lkm=('sairauspaaryhma', 'count'),
    saaja_lkm_mies=('sukupuoli', lambda x: (x == 'Mies').sum()),
    saaja_lkm_nainen=('sukupuoli', lambda x: (x == 'Nainen').sum()),
    saaja_lkm_muu=('sukupuoli', lambda x: ((x != 'Mies') & (x != 'Nainen')).sum())
).reset_index()


## ***2003-2023 Suomen työkyvyttömyyseläkkeensaajien määrä kunnittain:***


In [105]:
colormap = cm.LinearColormap(colors=['#81CAD6', '#EDCD44', '#DC3E26'], vmin=all_locations['saaja_lkm'].min(), vmax=all_locations['saaja_lkm'].max())

map = folium.Map(location=[64.9146659, 26.0672554], tiles='cartodb positron', zoom_start=5) 

for i in range(len(all_locations)):
    location = [all_locations.iloc[i]['latitude'], all_locations.iloc[i]['longitude']]
    hover_text = (
        f"<strong style='color:#996666'><u>{all_locations.iloc[i]['kunta_nimi']}</u></strong><br>"
        f"<i><small style='color:#808080'>{all_locations.iloc[i]['Maakunta']}</small></i><br>"
        f"<b><small style='color:#996666'>Total: {all_locations.iloc[i]['saaja_lkm']}</small></b><br>"
        f"<b><small style='color:#668099'>Mies: {all_locations.iloc[i]['saaja_lkm_mies']}</small></b><br>"
        f"<b><small style='color:#996699'>Nainen: {all_locations.iloc[i]['saaja_lkm_nainen']}</small></b><br>"
        f"<b><small style='color:#808080'>Muu: {all_locations.iloc[i]['saaja_lkm_muu']}</small></b><extra></extra>"
    )
    color = colormap(all_locations.iloc[i]['saaja_lkm'])
    
    folium.CircleMarker(
        location=location,
        radius=5,
        tooltip=hover_text,
        color=None, 
        fill=True,
        fill_color=color,  
        fill_opacity=0.9
    ).add_to(map)

colormap.add_to(map)

map.save('kunta_map.html')

map